# How to create custom dataset



In [1]:
import numpy
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data.dataloader as dataloader
import torch.optim as optim

from PIL import Image
from torch.utils.data.dataset import Dataset
from torch.utils.data import TensorDataset
from torch.autograd import Variable
from torchvision import transforms, datasets

import matplotlib.pyplot as plt

SEED = 1

# CUDA?
cuda = torch.cuda.is_available()

# For reproducibility
torch.manual_seed(SEED)

if cuda:
    torch.cuda.manual_seed(SEED)

In [15]:
transformations = transforms.Compose([transforms.Grayscale(),transforms.ToTensor()])
image_data = datasets.ImageFolder('PPC2',transformations)


train_size = int(0.8 * len(image_data))
test_size = len(image_data) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(image_data, [train_size, test_size])

print (train_dataset)
print (train_size)
print (test_size)

161
41


In [40]:
dataloader_args = dict(shuffle=True, batch_size=4,num_workers=4, pin_memory=True)
train_loader = dataloader.DataLoader(train_dataset, **dataloader_args)

dataloader_args = dict(shuffle=True, batch_size=len(test_dataset),num_workers=4, pin_memory=True)
test_loader = dataloader.DataLoader(test_dataset, **dataloader_args)




In [74]:
# One hidden Layer NN
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.fc0 = nn.Linear(32 * 32, 16*16)
        #self.fc1 = nn.Linear(1000, 3)
        self.fc1 = nn.Linear(16 * 16, 8*8)
        self.fc2 = nn.Linear(8*8, 3)
    def forward(self, x):
        #out = self.drop_out(out)
        x = x.view((-1, 1024))
        h = F.selu(self.fc0(x))
        h = F.selu(self.fc1(h))
        out = self.fc2(h)
        return F.log_softmax(out)    
    
    
model = Model()
if cuda:
    model.cuda()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [77]:
EPOCHS = 100
losses = []



dataiter = iter(test_loader)
images, labels = dataiter.next()

#model.eval()
print (images.shape)
#output = model(images)
    
train_size = len(train_loader.dataset)
batch_size = (train_size / 4) 


torch.Size([41, 1, 32, 32])


In [78]:

model.train()
for epoch in range(EPOCHS):
    for batch_idx, (data, target) in enumerate(train_loader):
        # Get Samples
        data, target = Variable(data), Variable(target)
        

        
        # Init
        optimizer.zero_grad()
        # Predict
        y_pred = model(data) 

         
        # Calculate loss
        loss = F.cross_entropy(y_pred, target)
        losses.append(loss.cpu().item())
        # Backpropagation
        loss.backward()
        optimizer.step()
        
        
        # Display
        if batch_idx % 4 == 1:
            print('\r Train Epoch: {}/{} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch+1,
                EPOCHS,
                batch_idx * len(data), 
                train_size,
                100. * batch_idx / batch_size, 
                loss.cpu().item()), 
                end='')
            
    # display final evaluation for this epoch
    model.eval()
    output = model(images)
    pred = output.data.max(1)[1]
    d = pred.eq(labels.data).cpu()
    accuracy = d.sum().item()/d.size()[0]
    
    print('\r Train Epoch: {}/{} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\t Test Accuracy: {:.4f}%'.format(
        epoch+1,
        EPOCHS,
        train_size, 
        train_size,
        100. * batch_idx / batch_size, 
        loss.cpu().item(),
        accuracy*100,
        end=''))

c:\users\mamoun\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:15: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  from ipykernel import kernelapp as app


 Train Epoch: 1/100 [161/161 (99%)]	Loss: 0.695865	 Test Accuracy: 43.9024%
 Train Epoch: 2/100 [161/161 (99%)]	Loss: 1.210890	 Test Accuracy: 31.7073%
 Train Epoch: 3/100 [161/161 (99%)]	Loss: 1.260665	 Test Accuracy: 31.7073%
 Train Epoch: 4/100 [161/161 (99%)]	Loss: 1.212166	 Test Accuracy: 31.7073%
 Train Epoch: 5/100 [161/161 (99%)]	Loss: 0.064271	 Test Accuracy: 39.0244%
 Train Epoch: 6/100 [161/161 (99%)]	Loss: 0.700621	 Test Accuracy: 41.4634%
 Train Epoch: 7/100 [161/161 (99%)]	Loss: 1.010016	 Test Accuracy: 43.9024%
 Train Epoch: 8/100 [161/161 (99%)]	Loss: 0.341729	 Test Accuracy: 48.7805%
 Train Epoch: 9/100 [161/161 (99%)]	Loss: 1.615328	 Test Accuracy: 51.2195%
 Train Epoch: 10/100 [161/161 (99%)]	Loss: 0.733485	 Test Accuracy: 41.4634%
 Train Epoch: 11/100 [161/161 (99%)]	Loss: 0.891465	 Test Accuracy: 53.6585%
 Train Epoch: 12/100 [161/161 (99%)]	Loss: 0.438790	 Test Accuracy: 31.7073%
 Train Epoch: 13/100 [161/161 (99%)]	Loss: 0.014688	 Test Accuracy: 36.5854%
 Train E